In [18]:
import pandas as pd
import numpy as np
import os

In [56]:
file_name = ['heart-rate-raw.csv','band-walk-raw.csv','band-run-raw.csv','band-bike-raw.csv','band-sit-raw.csv','band-syn-raw.csv']
path = os.getcwd()

对心率数据的格式转换

In [60]:
df_heart=pd.read_csv(path+'\\'+file_name[0])

# 将dateTime列转换为datetime格式
df_heart['dateTime'] = pd.to_datetime(df_heart['dateTime'], format='%d.%m.%Y %H:%M:%S')
# 将rateZone列转换为浮点数
df_heart['rateZone'] = df_heart['rateZone'].str.rstrip('%').astype(float) / 100

print(df_heart.dtypes)

df_heart = df_heart.sort_values('dateTime')
df_heart.to_csv(path+'\\'+'heart-rate.csv',index=False)

dateTime    datetime64[ns]
rate                 int64
rateZone           float64
dtype: object


对加速度传感器的格式转换,并合并心率

In [106]:
for i in range(1,6):
    df = pd.read_csv(path+'\\'+file_name[i])
    df = df.iloc[:,0].str.split(';', expand=True)
    df.columns = ['dateTime', 'BandAccX', 'BandAccY', 'BandAccZ']
    df['dateTime'] = pd.to_datetime(df['dateTime'].astype('int64'), unit='ms')
    df['dateTime'] = df['dateTime'] + pd.DateOffset(hours=2)
    df['BandAccX'] = df['BandAccX'].astype('float64')
    df['BandAccY'] = df['BandAccY'].astype('float64')
    df['BandAccZ'] = df['BandAccZ'].astype('float64')
    df = df.groupby('dateTime').mean().reset_index()
    df = df.sort_values('dateTime')
    print(file_name[i])
    print(df.head(5))

    
    print(df_heart.head(5))
    result_left = pd.merge_asof(df, df_heart, on='dateTime', direction='nearest',tolerance=pd.Timedelta('60s'))
    print(result_left.head(5))
    result_left.to_csv(path+'\\'+file_name[i].replace("-raw.csv","")+'.csv',index=False)

 



band-walk-raw.csv
                 dateTime  BandAccX     BandAccY     BandAccZ
0 2024-06-06 11:48:12.529   -1755.4  3500.200000 -1207.000000
1 2024-06-06 11:48:12.709   -1781.8  3567.800000 -1245.200000
2 2024-06-06 11:48:12.905   -1778.4  3496.200000 -1227.600000
3 2024-06-06 11:48:13.099   -1790.6  3504.600000 -1205.800000
4 2024-06-06 11:48:13.318   -1744.0  3488.166667  -994.666667
                dateTime  rate  rateZone
1293 2024-06-06 10:31:48    84      0.44
1292 2024-06-06 10:31:49    84      0.44
1291 2024-06-06 10:31:58    84      0.44
1290 2024-06-06 10:32:04    83      0.43
1289 2024-06-06 10:32:10    82      0.43
                 dateTime  BandAccX     BandAccY     BandAccZ  rate  rateZone
0 2024-06-06 11:48:12.529   -1755.4  3500.200000 -1207.000000    80      0.42
1 2024-06-06 11:48:12.709   -1781.8  3567.800000 -1245.200000    80      0.42
2 2024-06-06 11:48:12.905   -1778.4  3496.200000 -1227.600000    80      0.42
3 2024-06-06 11:48:13.099   -1790.6  3504.600000 -12